# Experiment 008: Small N Value Optimization

Focus on optimizing small N values (N=1-20) which contribute disproportionately to the total score.
- N=1 contributes 0.661 (almost 1% of total)
- Worst 20 N values contribute 8.08 points (11.43% of total)
- Improving them by 20% would save 1.62 points

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import time
from itertools import product

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [4]:
# Load current best submission
print("Loading current best submission...")
df = pd.read_csv('/home/submission/submission.csv')

# Calculate current scores for all N
current_scores = {}
current_configs = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    current_configs[n] = trees
    current_scores[n] = get_score(trees, n)

current_total = sum(current_scores.values())
print(f"Current total score: {current_total:.6f}")

# Show worst 20 N values
sorted_scores = sorted(current_scores.items(), key=lambda x: x[1], reverse=True)
print("\nWorst 20 N values (highest per-N scores):")
for n, score in sorted_scores[:20]:
    print(f"  N={n}: {score:.6f}")

Loading current best submission...


Current total score: 70.676104

Worst 20 N values (highest per-N scores):
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.379203
  N=10: 0.376630
  N=21: 0.376451
  N=20: 0.376057
  N=11: 0.375736
  N=22: 0.375258
  N=16: 0.374128
  N=26: 0.373997
  N=12: 0.372724
  N=13: 0.372323
  N=25: 0.372144


In [5]:
# For N=1, the optimal configuration is a single tree at angle 45 degrees
# This minimizes the bounding box (tree is symmetric at 45 degrees)

def optimize_n1():
    """For N=1, find the optimal rotation angle."""
    best_score = float('inf')
    best_angle = 0
    
    # Try many angles
    for angle in np.linspace(0, 360, 3601):  # 0.1 degree resolution
        tree = ChristmasTree('0', '0', str(angle))
        score = get_score([tree], 1)
        if score < best_score:
            best_score = score
            best_angle = angle
    
    return [ChristmasTree('0', '0', str(best_angle))], best_score, best_angle

print("Optimizing N=1...")
n1_trees, n1_score, n1_angle = optimize_n1()
print(f"N=1: Current score = {current_scores[1]:.6f}")
print(f"N=1: Optimal score = {n1_score:.6f} at angle {n1_angle:.1f}")
print(f"N=1: Improvement = {current_scores[1] - n1_score:.6f}")

Optimizing N=1...


N=1: Current score = 0.661250
N=1: Optimal score = 0.661250 at angle 45.0
N=1: Improvement = -0.000000


In [6]:
# For N=2, try all combinations of angles and positions
# Two trees can be placed side by side or one above the other

def optimize_n2(n_angle_steps=36, n_position_steps=20):
    """For N=2, find optimal configuration by grid search."""
    best_score = float('inf')
    best_config = None
    
    # Try different angle combinations
    angles = np.linspace(0, 360, n_angle_steps + 1)[:-1]
    
    for angle1 in angles:
        for angle2 in angles:
            # Create trees at origin first
            tree1 = ChristmasTree('0', '0', str(angle1))
            tree2 = ChristmasTree('0', '0', str(angle2))
            
            # Get bounding boxes
            b1 = tree1.polygon.bounds
            b2 = tree2.polygon.bounds
            
            # Try placing tree2 to the right of tree1
            for dx in np.linspace(0.3, 1.5, n_position_steps):
                for dy in np.linspace(-0.5, 0.5, n_position_steps):
                    tree2_test = ChristmasTree(str(dx), str(dy), str(angle2))
                    trees = [tree1, tree2_test]
                    
                    if not has_collision(trees):
                        score = get_score(trees, 2)
                        if score < best_score:
                            best_score = score
                            best_config = [tree1.clone(), tree2_test.clone()]
    
    return best_config, best_score

print("Optimizing N=2 (this may take a while)...")
start = time.time()
n2_trees, n2_score = optimize_n2(n_angle_steps=18, n_position_steps=15)
elapsed = time.time() - start
print(f"N=2: Current score = {current_scores[2]:.6f}")
print(f"N=2: Optimal score = {n2_score:.6f}")
print(f"N=2: Improvement = {current_scores[2] - n2_score:.6f}")
print(f"Time: {elapsed:.1f}s")

Optimizing N=2 (this may take a while)...


N=2: Current score = 0.450779
N=2: Optimal score = 0.544445
N=2: Improvement = -0.093666
Time: 8.5s


In [ ]:
# Extended SA optimization for small N values
# Use many restarts and longer runs

def extended_sa_optimization(n, trees, n_restarts=20, n_steps=20000, seed=42):
    """Extended SA with many restarts for small N values."""
    best_trees = [t.clone() for t in trees]
    best_score = get_score(best_trees, n)
    
    for restart in range(n_restarts):
        random.seed(seed + restart * 1000)
        
        # Start from current best with small perturbation
        current_trees = [t.clone() for t in best_trees]
        
        # Add random perturbation for restart
        if restart > 0:
            for i in range(len(current_trees)):
                dx = random.uniform(-0.1, 0.1)
                dy = random.uniform(-0.1, 0.1)
                da = random.uniform(-30, 30)
                new_x = float(current_trees[i].center_x) + dx
                new_y = float(current_trees[i].center_y) + dy
                new_a = (float(current_trees[i].angle) + da) % 360
                current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(new_a))
            
            # If perturbation caused collision, skip this restart
            if has_collision(current_trees):
                continue
        
        current_score = get_score(current_trees, n)
        
        # SA parameters
        T_max = 0.5
        T_min = 0.00001
        alpha = (T_min / T_max) ** (1.0 / n_steps)
        T = T_max
        
        for step in range(n_steps):
            # Pick a random tree
            i = random.randint(0, n - 1)
            
            # Save old state
            old_x = current_trees[i].center_x
            old_y = current_trees[i].center_y
            old_angle = current_trees[i].angle
            
            # Perturb with temperature-dependent scale
            scale = T / T_max
            dx = Decimal(str(random.uniform(-0.05 * scale, 0.05 * scale)))
            dy = Decimal(str(random.uniform(-0.05 * scale, 0.05 * scale)))
            dangle = Decimal(str(random.uniform(-20 * scale, 20 * scale)))
            
            new_x = old_x + dx
            new_y = old_y + dy
            new_angle = (old_angle + dangle) % Decimal('360')
            
            # Apply perturbation
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(new_angle))
            
            # Check collision
            if has_collision(current_trees):
                # Revert
                current_trees[i] = ChristmasTree(str(old_x), str(old_y), str(old_angle))
            else:
                new_score = get_score(current_trees, n)
                delta = new_score - current_score
                
                if delta < 0 or random.random() < math.exp(-delta / T):
                    current_score = new_score
                    if new_score < best_score:
                        best_score = new_score
                        best_trees = [t.clone() for t in current_trees]
                else:
                    # Revert
                    current_trees[i] = ChristmasTree(str(old_x), str(old_y), str(old_angle))
            
            T *= alpha
    
    return best_trees, best_score

print("Extended SA function defined")

In [ ]:
# Run extended SA on small N values (N=1 to 20)
print("Running extended SA on small N values...")

improved_configs = {}
improved_scores = {}
total_improvement = 0

for n in range(1, 21):
    trees = current_configs[n]
    original_score = current_scores[n]
    
    # More restarts and steps for smaller N
    n_restarts = max(5, 25 - n)
    n_steps = max(10000, 30000 - n * 1000)
    
    start = time.time()
    new_trees, new_score = extended_sa_optimization(n, trees, n_restarts=n_restarts, n_steps=n_steps, seed=42+n)
    elapsed = time.time() - start
    
    improved_configs[n] = new_trees
    improved_scores[n] = new_score
    
    improvement = original_score - new_score
    total_improvement += improvement
    
    if improvement > 0:
        print(f"  N={n}: {original_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
    else:
        print(f"  N={n}: {original_score:.6f} (no improvement) [{elapsed:.1f}s]")

print(f"\nTotal improvement from small N optimization: {total_improvement:.6f} points")

In [ ]:
# Create final configuration with improvements
print("\nCreating final configuration...")

final_configs = {}
final_scores = {}

for n in range(1, 201):
    if n <= 20 and n in improved_configs:
        # Use improved configuration if better
        if improved_scores[n] < current_scores[n]:
            final_configs[n] = improved_configs[n]
            final_scores[n] = improved_scores[n]
        else:
            final_configs[n] = current_configs[n]
            final_scores[n] = current_scores[n]
    else:
        final_configs[n] = current_configs[n]
        final_scores[n] = current_scores[n]

# Calculate final total
final_total = sum(final_scores.values())
print(f"Final total score: {final_total:.6f}")
print(f"Improvement over current: {current_total - final_total:.6f} points")
print(f"Gap to target: {final_total - 68.922808:.6f} points")

In [ ]:
# Verify no overlaps
print("\nVerifying no overlaps...")
overlap_count = 0
for n in range(1, 201):
    if has_collision(final_configs[n]):
        overlap_count += 1
        print(f"  N={n}: OVERLAP!")

print(f"Total overlaps: {overlap_count}")

In [ ]:
# Save final submission
print("\nSaving final submission...")

rows = []
for n in range(1, 201):
    trees = final_configs[n]
    for i, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })

final_df = pd.DataFrame(rows)
final_df.to_csv('/home/code/experiments/008_small_n_optimization/submission.csv', index=False)
final_df.to_csv('/home/submission/submission.csv', index=False)

print(f"Saved submission with score {final_total:.6f}")
print(f"Sample rows:")
print(final_df.head())